In [1]:
import pandas as pd
import lightwood as lw

INFO:lightwood-84095:No torchvision detected, image helpers not supported.
INFO:lightwood-84095:No torchvision/pillow detected, image encoder not supported
/home/belucci/miniconda3/envs/lightwood/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load Kaggle dataset (already downloaded/unzipped)
df = pd.read_csv("/home/belucci/downloads/winemag-data-130k-v2.csv")

# Keep only the columns we care about
df = df[["description", "price", "province", "points", "variety"]].dropna()

In [11]:
problem_definition = {
	"target": "variety",
	"dtype_dict": dict(variety="categorical"),
	"embedding_only": True,
}

In [12]:
json_ai = lw.json_ai_from_problem(df, problem_definition)

INFO:type_infer-84095:Analyzing a sample of 16939
INFO:type_infer-84095:from a total population of 120915, this is equivalent to 14.0% of your data.
INFO:type_infer-84095:Using 5 processes to deduct types.


INFO:type_infer-84095:Infering type for: price
INFO:type_infer-84095:Infering type for: province
INFO:type_infer-84095:Column price has data type integer
INFO:type_infer-84095:Infering type for: points
INFO:type_infer-84095:Infering type for: description
INFO:type_infer-84095:Infering type for: variety
INFO:type_infer-84095:Column points has data type integer
INFO:type_infer-84095:Doing text detection for column: province
INFO:type_infer-84095:Doing text detection for column: variety
INFO:type_infer-84095:Doing text detection for column: description
INFO:dataprep_ml-84095:Starting statistical analysis
INFO:dataprep_ml-84095:Finished statistical analysis


In [13]:
json_ai.to_dict()

{'encoders': {'variety': {'module': 'OneHotEncoder',
   'args': {'is_target': 'True',
    'target_weights': '$statistical_analysis.target_weights'}},
  'description': {'module': 'PretrainedLangEncoder',
   'args': {'output_type': '$dtype_dict[$target]',
    'stop_after': '$problem_definition.seconds_per_encoder'}},
  'price': {'module': 'NumericEncoder', 'args': {}},
  'province': {'module': 'CategoricalAutoEncoder',
   'args': {'stop_after': '$problem_definition.seconds_per_encoder'}},
  'points': {'module': 'NumericEncoder', 'args': {}}},
 'dtype_dict': {'description': 'rich_text',
  'price': 'integer',
  'province': 'short_text',
  'points': 'integer',
  'variety': 'categorical'},
 'dependency_dict': {},
 'model': {'module': 'BestOf',
  'args': {'submodels': [{'module': 'Neural',
     'args': {'fit_on_dev': True,
      'stop_after': '$problem_definition.seconds_per_mixer',
      'search_hyperparameters': True}},
    {'module': 'XGBoostMixer',
     'args': {'stop_after': '$problem_de

In [14]:
code = lw.code_from_json_ai(json_ai)

In [15]:
print(code)

import lightwood
from lightwood import __version__ as lightwood_version
from lightwood.analysis import *
from lightwood.api import *
from lightwood.data import *
from lightwood.encoder import *
from lightwood.ensemble import *
from lightwood.helpers.device import *
from lightwood.helpers.general import *
from lightwood.helpers.ts import *
from lightwood.helpers.log import *
from lightwood.helpers.numeric import *
from lightwood.helpers.parallelism import *
from lightwood.helpers.seed import *
from lightwood.helpers.text import *
from lightwood.helpers.torch import *
from lightwood.mixer import *

from dataprep_ml.insights import statistical_analysis
from dataprep_ml.cleaners import cleaner
from dataprep_ml.splitters import splitter
from dataprep_ml.imputers import *

from mindsdb_evaluator import evaluate_accuracies
from mindsdb_evaluator.accuracy import __all__ as mdb_eval_accuracy_metrics

import pandas as pd
from typing import Dict, List, Union, Optional
import os
from types import 

In [16]:
predictor = lw.predictor_from_code(code)

In [17]:
predictor.learn(df)

INFO:dataprep_ml-84095:[Learn phase 1/8] - Statistical analysis
INFO:dataprep_ml-84095:Starting statistical analysis
INFO:dataprep_ml-84095:Finished statistical analysis
DEBUG:lightwood-84095: `analyze_data` runtime: 1.41 seconds
INFO:dataprep_ml-84095:[Learn phase 2/8] - Data preprocessing
INFO:dataprep_ml-84095:Cleaning the data
DEBUG:lightwood-84095: `preprocess` runtime: 0.38 seconds
INFO:dataprep_ml-84095:[Learn phase 3/8] - Data splitting
INFO:dataprep_ml-84095:Splitting the data into train/test
/home/belucci/miniconda3/envs/lightwood/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
DEBUG:lightwood-84095: `split` runtime: 0.86 seconds
INFO:dataprep_ml-84095:[Learn phase 4/8] - Preparing encoders
INFO:lightwood-84095:Embedding mode on. [CLS] embedding dim output of encode()
DEBUG:dataprep_ml-84095:Preparing seq

In [19]:
embeded_dataset = predictor.predict(df)

INFO:dataprep_ml-84095:[Predict phase 1/4] - Data preprocessing
INFO:dataprep_ml-84095:Cleaning the data
DEBUG:lightwood-84095: `preprocess` runtime: 0.47 seconds
INFO:dataprep_ml-84095:[Predict phase 2/4] - Feature generation
INFO:dataprep_ml-84095:Featurizing the data
DEBUG:lightwood-84095: `featurize` runtime: 1716.78 seconds
INFO:dataprep_ml-84095:[Predict phase 3/4] - Calling ensemble
DEBUG:lightwood-84095: `_timed_call` runtime: 0.09 seconds
DEBUG:lightwood-84095: `predict` runtime: 1717.61 seconds


In [21]:
embeded_dataset.to_csv("/home/belucci/downloads/winemag-data-130k-v2-embeded.csv")

In [27]:
type(embeded_dataset)

pandas.core.frame.DataFrame

In [28]:
# Check predictor attributes and methods
print("Predictor attributes:")
for attr in dir(predictor):
    if not attr.startswith('_'):
        print(f"  {attr}")
        
print("\n" + "="*50)
print("JSON AI structure (feature information):")
print(json_ai.to_dict().keys())

Predictor attributes:
  accuracy_functions
  adjust
  analysis_blocks
  analyze_data
  analyze_ensemble
  dependencies
  dtype_dict
  encoders
  ensemble
  export
  feature_cache
  featurize
  fit
  fit_mixer
  global_insights
  identifiers
  imputers
  input_cols
  learn
  lightwood_version
  mixers
  mode
  model_analysis
  pred_args
  predict
  prepare
  preprocess
  problem_definition
  runtime_analyzer
  runtime_log
  save
  split
  statistical_analysis
  supports_proba
  target
  test
  ts_analysis

JSON AI structure (feature information):
dict_keys(['encoders', 'dtype_dict', 'dependency_dict', 'model', 'problem_definition', 'identifiers', 'imputers', 'accuracy_functions'])


In [29]:
# Check the structure of the embedded dataset
print("Embedded dataset shape:", embeded_dataset.shape)
print("\nEmbedded dataset columns:")
print(list(embeded_dataset.columns))

print("\n" + "="*50)
print("Original dataset columns:")
print(list(df.columns))

print("\n" + "="*50)
print("Input columns from predictor:")
print(predictor.input_cols)

Embedded dataset shape: (120915, 876)

Embedded dataset columns:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208,

In [30]:
# Explore the encoders to understand feature mappings
print("Encoders information:")
for col, encoder in predictor.encoders.items():
    print(f"\nColumn: {col}")
    print(f"  Encoder type: {type(encoder).__name__}")
    
    # Check if encoder has output_size or similar attributes
    if hasattr(encoder, 'output_size'):
        print(f"  Output size: {encoder.output_size}")
    if hasattr(encoder, 'vocab_size'):
        print(f"  Vocab size: {encoder.vocab_size}")
    if hasattr(encoder, 'embed_dims'):
        print(f"  Embedding dimensions: {encoder.embed_dims}")
    
    # Print all non-private attributes
    attrs = [attr for attr in dir(encoder) if not attr.startswith('_')]
    print(f"  Available attributes: {attrs[:10]}...")  # Show first 10 to avoid too much output

Encoders information:

Column: variety
  Encoder type: OneHotEncoder
  Output size: 692
  Available attributes: ['decode', 'decode_probabilities', 'dependencies', 'encode', 'index_weights', 'is_prepared', 'is_target', 'is_timeseries_encoder', 'is_trainable_encoder', 'map']...

Column: description
  Encoder type: PretrainedLangEncoder
  Output size: 768
  Available attributes: ['decode', 'dependencies', 'device', 'embed_mode', 'encode', 'is_prepared', 'is_target', 'is_timeseries_encoder', 'is_trainable_encoder', 'model_type']...

Column: price
  Encoder type: NumericEncoder
  Output size: 4
  Available attributes: ['decode', 'decode_log', 'dependencies', 'encode', 'get_weights', 'index_weights', 'is_prepared', 'is_target', 'is_timeseries_encoder', 'is_trainable_encoder']...

Column: province
  Encoder type: CategoricalAutoEncoder
  Output size: 100
  Available attributes: ['batch_size', 'decode', 'decoder', 'dependencies', 'desired_error', 'device_type', 'encode', 'encoder', 'input_enco

In [31]:
# Create a feature mapping dictionary
feature_mapping = {}
start_col = 0

# Order of processing (based on input_cols + target)
processing_order = predictor.input_cols + [predictor.target]
print("Processing order:", processing_order)
print("\nFeature Column Mapping:")
print("=" * 60)

for col in processing_order:
    encoder = predictor.encoders[col]
    output_size = encoder.output_size
    end_col = start_col + output_size
    
    feature_mapping[col] = {
        'columns': list(range(start_col, end_col)),
        'column_range': f"{start_col}-{end_col-1}",
        'size': output_size,
        'encoder_type': type(encoder).__name__
    }
    
    print(f"'{col}':")
    print(f"  Columns: {start_col} to {end_col-1} ({output_size} features)")
    print(f"  Encoder: {type(encoder).__name__}")
    print(f"  Description: ", end="")
    
    if isinstance(encoder.__class__.__name__, str):
        if "OneHot" in encoder.__class__.__name__:
            print("One-hot encoded categorical values")
        elif "PretrainedLang" in encoder.__class__.__name__:
            print("Text embeddings from pretrained language model")
        elif "Numeric" in encoder.__class__.__name__:
            print("Numerical features (normalized, binned, etc.)")
        elif "CategoricalAuto" in encoder.__class__.__name__:
            print("Categorical autoencoder embeddings")
        else:
            print("Unknown encoder type")
    print()
    
    start_col = end_col

print(f"Total features: {start_col}")
print(f"Embedded dataset shape: {embeded_dataset.shape}")

# Store the mapping for future use
print("\n" + "="*60)
print("Feature mapping dictionary:")
for col, info in feature_mapping.items():
    print(f"'{col}': columns {info['column_range']} ({info['size']} features, {info['encoder_type']})")

Processing order: ['description', 'price', 'province', 'points', 'variety']

Feature Column Mapping:
'description':
  Columns: 0 to 767 (768 features)
  Encoder: PretrainedLangEncoder
  Description: Text embeddings from pretrained language model

'price':
  Columns: 768 to 771 (4 features)
  Encoder: NumericEncoder
  Description: Numerical features (normalized, binned, etc.)

'province':
  Columns: 772 to 871 (100 features)
  Encoder: CategoricalAutoEncoder
  Description: Categorical autoencoder embeddings

'points':
  Columns: 872 to 875 (4 features)
  Encoder: NumericEncoder
  Description: Numerical features (normalized, binned, etc.)

'variety':
  Columns: 876 to 1567 (692 features)
  Encoder: OneHotEncoder
  Description: One-hot encoded categorical values

Total features: 1568
Embedded dataset shape: (120915, 876)

Feature mapping dictionary:
'description': columns 0-767 (768 features, PretrainedLangEncoder)
'price': columns 768-771 (4 features, NumericEncoder)
'province': columns 

In [32]:
# Let's check if the target column (variety) is actually included in the embedded dataset
print("Problem definition:")
print(problem_definition)
print("\nEmbedding only mode:", problem_definition.get('embedding_only', False))

# Since embedding_only=True, the target might not be included in embeddings
# Let's recalculate without the target
input_only_mapping = {}
start_col = 0

print("\nActual embedded features (input columns only):")
print("=" * 60)

for col in predictor.input_cols:  # Only input columns, not target
    encoder = predictor.encoders[col]
    output_size = encoder.output_size
    end_col = start_col + output_size
    
    input_only_mapping[col] = {
        'columns': list(range(start_col, end_col)),
        'column_range': f"{start_col}-{end_col-1}",
        'size': output_size,
        'encoder_type': type(encoder).__name__
    }
    
    print(f"'{col}':")
    print(f"  Columns: {start_col} to {end_col-1} ({output_size} features)")
    print(f"  Encoder: {type(encoder).__name__}")
    print()
    
    start_col = end_col

print(f"Total input features: {start_col}")
print(f"Embedded dataset actual shape: {embeded_dataset.shape}")

# Check a few sample values to understand the structure better
print(f"\nFirst few values of embedded dataset:")
print(embeded_dataset.iloc[0, :10])  # First row, first 10 columns

Problem definition:
{'target': 'variety', 'dtype_dict': {'variety': 'categorical'}, 'embedding_only': True}

Embedding only mode: True

Actual embedded features (input columns only):
'description':
  Columns: 0 to 767 (768 features)
  Encoder: PretrainedLangEncoder

'price':
  Columns: 768 to 771 (4 features)
  Encoder: NumericEncoder

'province':
  Columns: 772 to 871 (100 features)
  Encoder: CategoricalAutoEncoder

'points':
  Columns: 872 to 875 (4 features)
  Encoder: NumericEncoder

Total input features: 876
Embedded dataset actual shape: (120915, 876)

First few values of embedded dataset:
0    0.934224
1    1.296476
2    0.430697
3    0.221121
4    1.998642
5    0.726594
6    1.706069
7    0.364600
8   -0.097179
9   -0.292101
Name: 0, dtype: float32


In [33]:
# Final Feature Mapping Summary
print("🎯 FEATURE MAPPING SUMMARY")
print("=" * 60)
print(f"Original dataset: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"Embedded dataset: {embeded_dataset.shape[0]} rows × {embeded_dataset.shape[1]} columns")
print(f"Target column '{predictor.target}' excluded due to embedding_only=True")
print()

# Create the final mapping dictionary for easy access
final_feature_mapping = {
    'description': {'start': 0, 'end': 767, 'size': 768, 'type': 'Text embeddings'},
    'price': {'start': 768, 'end': 771, 'size': 4, 'type': 'Numerical features'},
    'province': {'start': 772, 'end': 871, 'size': 100, 'type': 'Categorical embeddings'}, 
    'points': {'start': 872, 'end': 875, 'size': 4, 'type': 'Numerical features'}
}

for feature, mapping in final_feature_mapping.items():
    print(f"📊 {feature.upper()}:")
    print(f"   Columns: {mapping['start']} to {mapping['end']} ({mapping['size']} features)")
    print(f"   Type: {mapping['type']}")
    print()

print("💡 How to access specific features:")
print("   Description embeddings: embeded_dataset.iloc[:, 0:768]")
print("   Price features: embeded_dataset.iloc[:, 768:772]") 
print("   Province embeddings: embeded_dataset.iloc[:, 772:872]")
print("   Points features: embeded_dataset.iloc[:, 872:876]")

🎯 FEATURE MAPPING SUMMARY
Original dataset: 120915 rows × 5 columns
Embedded dataset: 120915 rows × 876 columns
Target column 'variety' excluded due to embedding_only=True

📊 DESCRIPTION:
   Columns: 0 to 767 (768 features)
   Type: Text embeddings

📊 PRICE:
   Columns: 768 to 771 (4 features)
   Type: Numerical features

📊 PROVINCE:
   Columns: 772 to 871 (100 features)
   Type: Categorical embeddings

📊 POINTS:
   Columns: 872 to 875 (4 features)
   Type: Numerical features

💡 How to access specific features:
   Description embeddings: embeded_dataset.iloc[:, 0:768]
   Price features: embeded_dataset.iloc[:, 768:772]
   Province embeddings: embeded_dataset.iloc[:, 772:872]
   Points features: embeded_dataset.iloc[:, 872:876]


In [34]:
# Create meaningful column names for the embedded dataset
column_names = []

# Generate column names based on the feature mapping
for feature, mapping in final_feature_mapping.items():
    feature_size = mapping['size']
    
    if feature == 'description':
        # Text embeddings: description_emb_0, description_emb_1, ...
        column_names.extend([f"description_emb_{i}" for i in range(feature_size)])
    elif feature == 'price':
        # Numerical features: price_feat_0, price_feat_1, ...
        column_names.extend([f"price_feat_{i}" for i in range(feature_size)])
    elif feature == 'province':
        # Categorical embeddings: province_emb_0, province_emb_1, ...
        column_names.extend([f"province_emb_{i}" for i in range(feature_size)])
    elif feature == 'points':
        # Numerical features: points_feat_0, points_feat_1, ...
        column_names.extend([f"points_feat_{i}" for i in range(feature_size)])

print(f"Created {len(column_names)} column names")
print("Sample column names:")
print("First 10:", column_names[:10])
print("Around price boundary:", column_names[765:775])
print("Around province boundary:", column_names[870:880])
print("Last 10:", column_names[-10:])

Created 876 column names
Sample column names:
First 10: ['description_emb_0', 'description_emb_1', 'description_emb_2', 'description_emb_3', 'description_emb_4', 'description_emb_5', 'description_emb_6', 'description_emb_7', 'description_emb_8', 'description_emb_9']
Around price boundary: ['description_emb_765', 'description_emb_766', 'description_emb_767', 'price_feat_0', 'price_feat_1', 'price_feat_2', 'price_feat_3', 'province_emb_0', 'province_emb_1', 'province_emb_2']
Around province boundary: ['province_emb_98', 'province_emb_99', 'points_feat_0', 'points_feat_1', 'points_feat_2', 'points_feat_3']
Last 10: ['province_emb_94', 'province_emb_95', 'province_emb_96', 'province_emb_97', 'province_emb_98', 'province_emb_99', 'points_feat_0', 'points_feat_1', 'points_feat_2', 'points_feat_3']


In [35]:
# Create a copy of the embedded dataset with meaningful column names
embeded_dataset_named = embeded_dataset.copy()
embeded_dataset_named.columns = column_names

print("Dataset with renamed columns:")
print(f"Shape: {embeded_dataset_named.shape}")
print("\nColumn names by feature group:")
print("Description embeddings (first 5):", list(embeded_dataset_named.columns[0:5]))
print("Price features:", list(embeded_dataset_named.columns[768:772]))
print("Province embeddings (first 5):", list(embeded_dataset_named.columns[772:777]))
print("Points features:", list(embeded_dataset_named.columns[872:876]))

# Display a sample of the renamed dataset
print("\nSample of renamed dataset:")
print(embeded_dataset_named.iloc[:3, [0, 1, 768, 769, 772, 773, 872, 873]])

Dataset with renamed columns:
Shape: (120915, 876)

Column names by feature group:
Description embeddings (first 5): ['description_emb_0', 'description_emb_1', 'description_emb_2', 'description_emb_3', 'description_emb_4']
Price features: ['price_feat_0', 'price_feat_1', 'price_feat_2', 'price_feat_3']
Province embeddings (first 5): ['province_emb_0', 'province_emb_1', 'province_emb_2', 'province_emb_3', 'province_emb_4']
Points features: ['points_feat_0', 'points_feat_1', 'points_feat_2', 'points_feat_3']

Sample of renamed dataset:
   description_emb_0  description_emb_1  price_feat_0  price_feat_1  \
0           0.934224           1.296476           1.0      2.708050   
1          -0.045748           1.364022           1.0      2.639057   
2          -0.042107           0.730143           1.0      2.564949   

   province_emb_0  province_emb_1  points_feat_0  points_feat_1  
0       -0.649276        0.241552            1.0       4.465908  
1       -0.713199       -0.653497          

In [36]:
# Save the renamed embedded dataset to CSV
output_file = "/home/belucci/downloads/winemag-data-130k-v2-embedded-named.csv"
embeded_dataset_named.to_csv(output_file, index=False)

print(f"✅ Saved renamed embedded dataset to: {output_file}")
print(f"📊 File contains {embeded_dataset_named.shape[0]} rows and {embeded_dataset_named.shape[1]} columns")

# Also create a feature mapping reference file
mapping_file = "/home/belucci/downloads/winemag-feature-mapping.txt"
with open(mapping_file, 'w') as f:
    f.write("Wine Dataset Feature Mapping\n")
    f.write("=" * 50 + "\n\n")
    f.write(f"Original dataset: {df.shape[0]} rows × {df.shape[1]} columns\n")
    f.write(f"Embedded dataset: {embeded_dataset_named.shape[0]} rows × {embeded_dataset_named.shape[1]} columns\n")
    f.write(f"Target column '{predictor.target}' excluded due to embedding_only=True\n\n")
    
    for feature, mapping in final_feature_mapping.items():
        f.write(f"{feature.upper()}:\n")
        f.write(f"  Columns: {mapping['start']} to {mapping['end']} ({mapping['size']} features)\n")
        f.write(f"  Type: {mapping['type']}\n")
        f.write(f"  Column names: {feature}_emb_0 to {feature}_emb_{mapping['size']-1}\n\n")

print(f"📝 Saved feature mapping reference to: {mapping_file}")

# Quick verification
print(f"\n🔍 Quick verification:")
print(f"  CSV file size: ~{embeded_dataset_named.memory_usage(deep=True).sum() / (1024*1024):.1f} MB in memory")
print(f"  Feature groups: {len(final_feature_mapping)} groups")
print(f"  Total features: {sum(m['size'] for m in final_feature_mapping.values())}")

✅ Saved renamed embedded dataset to: /home/belucci/downloads/winemag-data-130k-v2-embedded-named.csv
📊 File contains 120915 rows and 876 columns
📝 Saved feature mapping reference to: /home/belucci/downloads/winemag-feature-mapping.txt

🔍 Quick verification:
  CSV file size: ~404.1 MB in memory
  Feature groups: 4 groups
  Total features: 876
